In [442]:
import numpy as np
import pandas as pd
import random, sys, time, copy
import time
random.seed(10)

In [443]:
#Graph code
class Graph:
    def __init__(self, n, alpha):
        self.exps, self.pages, self.ints = n
        self.map = np.zeros(n)
        self.alpha = alpha
        
        for exp in range(0,self.exps):
            for page in range(0,self.pages):
                data = list((range(0, self.ints)))
                np.random.shuffle(data)
                self.map[exp,page] = data
        
        self.trail = np.full(n,self.alpha) #Initialize pheromone on trail
        self.ev = np.random.randint(1,50,n)  #Random generate engagement values
    
    def calc_score(self, exp, page, interaction):
        ev = self.ev[exp][page][interaction]
        pheromone = self.trail[exp][page][interaction]       
        return ev,pheromone
    
    def get_pheromone_page(self, exp, page):
        return self.trail[exp][page]
        
    def calc_affinity_score(self, exp, page, interaction):
        return self.ev[exp][page][interaction]

In [444]:
#ACO code
class ant:
    def __init__(self, graph):
        self.graph = graph
        self.map = graph.map
        self.trail = graph.trail
        self.score_list = []
        self.routelist = []
        self.exps = []
        self.total_score = 0
        self.pheromone_list = []
        
    def get_trail(self,map,random_prob):
        pages_score = 0
        pheromones = 0
        exp = random.randint(0,self.graph.exps-1) #Random experience will be pick
        route = []
        for page in range(self.graph.pages):
            if random.random()>(1-random_prob): #Determine whether go to a random route or the pheromone route
                interaction = random.randint(0,self.graph.map[exp][page].size-1)
            else:
                pheromone_page = self.graph.get_pheromone_page(exp,page)
                max_index = np.where(pheromone_page == max(pheromone_page))[0] #Get the interaction index with most pheromone deposited
                interaction = np.random.choice(max_index)

            score,pheromone = self.graph.calc_score(exp,page,interaction)
            pages_score += score
            pheromones += pheromone
            route.append(interaction)
        self.score_list.append(pages_score)
        self.pheromone_list.append(pheromones)
        self.routelist.append(route)
        self.exps.append(exp)
        self.total_score += pages_score
            
        return list(zip(self.exps, self.routelist, self.score_list, self.pheromone_list))

    
    def update_trail(self, routes):
        route_scores = np.array(routes)[:,2]    
        top1 = np.sort(route_scores)[-1:] #Get the top1 score

        for route in routes:
            exp,path,score,pheromone = route
            for idx, interaction in enumerate(path): 
                if(np.isin(score,top1)): #Update the pheromone trail with top1 score only
                    self.graph.trail[exp][idx][interaction] += (score / self.total_score)*0.1
        return
    

In [445]:
class colony:
    def __init__(self, graph, iterations, pop, best_score,random_prob):
        self.graph = graph
        self.pop = pop
        self.ants = []
        self.best_route = []
        self.best_score = best_score
        self.best_experience = 0
        self.iterations = iterations
        self.best_pheromone = 0
        self.random_prob = random_prob
   
        for i in range(0,pop):
            new_ant = ant(graph)
            self.ants.append(new_ant)
        
    def run(self):
        count = 0
        self.graph.trail = self.graph.trail / 2
        for iteration in range(self.iterations):
            for ant in self.ants:
                result = ant.get_trail(self.graph.map,random_prob)
                ant.update_trail(result)

                index = np.argmax(np.array(result)[:,2]) #Retrieve the index from highest score trail
                exp,route,score,pheromone = np.array(result)[index]  
            
                if score > self.best_score:
                    self.best_score = score
                    self.best_route = route
                    self.best_experience = exp
                    self.best_pheromone = pheromone/(self.iterations*self.pop) #average pheromone increased for each loop in this trail
                if score == self.best_score:
                    count+=1
            
        print ("best exp,route,score,pheromone: ", self.best_experience, self.best_route, self.best_score, self.best_pheromone)
        print ("route best match count: ", count)

In [446]:
initial_pheromone = 0.01
experiences = 1 #Personalization Experience count
web_pages = 10 #Web pages count
interactions = 10 #Interaction count
iterations = 50
populations = 300
best_score = 100
graph = Graph([experiences,web_pages,interactions],initial_pheromone) #Using one graph only

In [447]:
#Algorithm 2 (ACO) (f=0.5)
start_time = time.time()
random_prob = 0.5
new_colony = colony(graph,iterations,populations,best_score,random_prob)
new_colony.run()
print("--- %s seconds ---" % (time.time() - start_time))

best exp,route,score,pheremone:  0 [6, 0, 7, 1, 8, 0, 8, 8, 2, 5] 430 0.02255543621692587
route best match count:  60
--- 55.14320492744446 seconds ---


In [448]:
#Algorithm 2 (ACO) (f=1.0)
start_time = time.time()
random_prob = 1
new_colony = colony(graph,iterations,populations,best_score,random_prob)
new_colony.run()
print("--- %s seconds ---" % (time.time() - start_time))

best exp,route,score,pheremone:  0 [6, 2, 7, 9, 8, 0, 4, 2, 0, 1] 400 0.024701272586079456
route best match count:  54
--- 55.146278858184814 seconds ---


In [449]:
#AIS code
class bcell:
    def __init__(self, graph, max_affinity):
        self.graph = graph
        self.map = graph.map
        self.max_affinity = max_affinity
        self.current_affinity =0
        self.exp = 0
        self.current_path = self.new_path()
    
    def new_path(self):
        pages_score = 0
        self.exp = random.randint(0,self.graph.exps-1)
        route = []
        for page in range(self.graph.pages):
            interaction = random.randint(0,self.graph.map[self.exp][page].size-1)
            route.append(interaction)

        return route
               
        
    def calc_affinity(self,exp,page,interaction):
        return self.graph.calc_affinity_score(exp,page,interaction)
              
    def clone_self(self):
        new_copy = copy.deepcopy(self)
        return new_copy
         
    def move(self):
 
        while self.current_path != []:
            self.current_path = self.new_path()
            self.current_affinity = 0
            for idx, interaction in enumerate(self.current_path):
                self.current_affinity += self.calc_affinity(self.exp,idx,interaction)
 
            if self.current_affinity < self.max_affinity:
                clone = self.clone_self()
                clone.move                
            else:
                break
             
        return self.current_affinity, self.current_path

In [450]:
class immune:
    def __init__(self, graph, iterations, starting_cells, max_affinity):
        self.best_affinity =[]
        self.max_affinity = max_affinity
        self.iterations = iterations
        self.starting_cells = starting_cells
        self.path_list = []
        self.graph = graph
        self.cells = []  
     
            
    def run(self):
        for i in range(self.iterations):
            
            for j in range(0,self.starting_cells):
                new_cell = bcell(self.graph, self.max_affinity)
                self.cells.append(new_cell)
            
            for cell in self.cells:
                affinity,path = cell.move()
                self.path_list.append(path)
                self.best_affinity.append(affinity)
                
        max_index = np.argmax(self.best_affinity)
       
        print ("best affinity = ", self.best_affinity[max_index])
        print ("best path = ", self.path_list[max_index])

In [451]:
#Algorithm 3 (AIS)
start_time = time.time()
new_immune = immune(graph, iterations, populations, best_score)
new_immune.run()
print("--- %s seconds ---" % (time.time() - start_time))

best affinity =  431
best path =  [4, 2, 7, 3, 7, 0, 0, 8, 1, 2]
--- 17.072441816329956 seconds ---
